In [ ]:
from google.colab import files

In [ ]:
files.upload()

In [ ]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d biaiscience/dogs-vs-cats

In [ ]:
!unzip dogs-vs-cats.zip

In [ ]:
!ls 

In [ ]:
import pathlib
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
TRAIN_DIR = pathlib.Path('train/train')
TEST_DIR  = pathlib.Path('test/test')

In [ ]:
train_list_ds = tf.data.Dataset.list_files(str(TRAIN_DIR/'*.*.jpg'))
# test_list_ds = tf.data.Dataset.list_files(str(TEST_DIR/'*.*.jpg'))

In [ ]:
for f in train_list_ds.take(5):
  print(f.numpy())

In [ ]:
CLASS_NAMES = ['dog', 'cat']
IMAGE_HEIGHT = 160
IMAGE_WIDTH  = 160
IMAGE_SHAPE = (IMAGE_HEIGHT, IMAGE_WIDTH, 3)
BATCH_SIZE = 32

In [ ]:
def get_label(file_path):
  print(file_path)
  parts = tf.strings.split(file_path, '/')
  parts = tf.strings.split(parts[-1], '.')
  if parts[-3] == CLASS_NAMES[0]:
    return 0
  return 1
  print(parts[-3])

In [ ]:
def decode_img(img):
  img = tf.image.decode_jpeg(img, channels=3)
  img = tf.image.convert_image_dtype(img, tf.float32)
  # img = (img/127.5) - 1
  return tf.image.resize(img, [IMAGE_HEIGHT, IMAGE_WIDTH])

In [ ]:
def process_path(file_path):
  label = get_label(file_path=file_path)
  img   = tf.io.read_file(file_path)
  img   = decode_img(img)
  return img, label

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [ ]:
labeled_ds = train_list_ds.map(process_path, num_parallel_calls=AUTOTUNE)

In [ ]:
for image, label in labeled_ds.take(1):
  plt.imshow(image.numpy())
  plt.title(label.numpy())
  print("Image shape: ", image.numpy().shape)
  print("Label: ", label.numpy())

In [ ]:
def prepare_for_training(ds, cache=True, shuffle_buffer_size=1000):
  if cache:
    if isinstance(cache, str):
      ds = ds.cache(cache)
    else:
      ds = ds.cache()

  ds = ds.shuffle(buffer_size=shuffle_buffer_size)
  # ds = ds.repeat()

  ds = ds.batch(BATCH_SIZE)
  ds = ds.prefetch(buffer_size=AUTOTUNE)

  return ds

In [ ]:
train_ds = prepare_for_training(labeled_ds)

In [ ]:
image_batch, label_batch = next(iter(train_ds))

In [ ]:
plt.imshow(image_batch[1].numpy())
plt.title(label_batch[1].numpy())
print("Image shape: ", image_batch[0].numpy().shape)
print("Label: ", label_batch[0].numpy())

In [ ]:
from tensorflow.keras import Sequential, layers, losses, optimizers

In [ ]:
base_model = tf.keras.applications.MobileNetV2(input_shape=IMAGE_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

In [ ]:
feature_batch = base_model(image_batch)

In [ ]:
feature_batch.shape

In [ ]:
base_model.trainable = False

In [ ]:
base_model.summary()

In [ ]:
global_average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_pooling_layer(feature_batch)
print(feature_batch_average.shape)

In [ ]:
prediction_layer = layers.Dense(1)
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

In [ ]:
model = Sequential([
                    base_model,
                    global_average_pooling_layer,
                    prediction_layer
])

In [ ]:
model = Sequential([
                    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
                    layers.Conv2D(64, (3, 3), activation='relu'),
                    layers.MaxPool2D((2,2)),
                    layers.Conv2D(128, (3, 3), activation='relu'),
                    layers.MaxPool2D((2,2)),
                    layers.Flatten(),
                    layers.Dense(728, activation='relu'),
                    layers.Dense(512, activation='relu'),
                    # layers.Dense(128, activation='relu'),
                    # layers.Dense(10, activation='relu'),
                    layers.Dense(1)
])

In [ ]:
model.compile(loss=losses.BinaryCrossentropy(from_logits=True),metrics=['accuracy'], optimizer=optimizers.Adam(0.001))

In [ ]:
model.summary()

In [ ]:
model.fit(train_ds,epochs=10)

In [ ]:
model.predict()

In [ ]:
from PIL import Image
import numpy as np

In [ ]:
!ls

In [ ]:
image = Image.open('cat.jpg')

In [ ]:
image = image.resize((IMAGE_HEIGHT, IMAGE_WIDTH), resample=Image.NEAREST)

In [ ]:
image = np.array(image)

In [ ]:
image = image[None,:,:,:]

In [ ]:
image.shape

In [ ]:
model.predict(image)

In [ ]:
model.save_weights('my_model.h5')

In [ ]:
files.download('my_model.h5')